In [1]:
import numpy
import opensim as osim
import xml.etree.ElementTree as ET


In [2]:
modelFileName = 'Rajagopal_2015.osim'          # The path to an input model
orientationsFileName = 'MT_012005D6_009-001_orientations.sto'   # The path to orientation data for calibration
sensor_to_opensim_rotations = osim.Vec3(-1.570796, 0, 0)  # The rotation of IMU data to the OpenSim world frame
baseIMUName = 'pelvis_imu'                     # The base IMU is the IMU on the base body of the model that dictates the heading (forward) direction of the model.
baseIMUHeading = 'z'                           # The Coordinate Axis of the base IMU that points in the heading direction.
visulizeCalibration = False


imuPlacer = osim.IMUPlacer()

In [4]:
imuPlacer.set_model_file(modelFileName)
imuPlacer.set_orientation_file_for_calibration(orientationsFileName)
imuPlacer.set_sensor_to_opensim_rotations(sensor_to_opensim_rotations)
imuPlacer.set_base_imu_label(baseIMUName)
imuPlacer.set_base_heading_axis(baseIMUHeading)

In [4]:
imuPlacer.run()

RuntimeError: std::exception in 'bool OpenSim::IMUPlacer::run(bool)': No model or model_file was specified for IMUPlacer.
	Thrown at IMUPlacer.cpp:90 in run().

<opensim.simbody.Vec3; proxy of <Swig Object of type 'SimTK::Vec< 3 > *' at 0x0000017EF2F0EB70> >

In [5]:
model = osim.Model("Rajagopal_2015.osim")
muscles = model.getMuscleSet()
muscles.getSize()

AttributeError: 'Model' object has no attribute 'getMuscleSet'

In [16]:
for body in model.getBodySet():
    print(body.get_mass() )
    print(body.get_inertia())
print(body)

11.777
~[0.1028,0.0871,0.0579,0,0,0]
9.3014
~[0.1339,0.0351,0.1412,0,0,0]
3.7075
~[0.0504,0.0051,0.0511,0,0,0]
0.0862
~[2.87e-06,1.311e-05,1.311e-05,0,0,0]
0.1
~[0.001,0.001,0.001,0,0,0]
1.25
~[0.0014,0.0039,0.0041,0,0,0]
0.2166
~[0.0001,0.0002,0.001,0,0,0]
9.3014
~[0.1339,0.0351,0.1412,0,0,0]
3.7075
~[0.0504,0.0051,0.0511,0,0,0]
0.0862
~[2.87e-06,1.311e-05,1.311e-05,0,0,0]
0.1
~[0.001,0.001,0.001,0,0,0]
1.25
~[0.0014,0.0039,0.0041,0,0,0]
0.2166
~[0.0001,0.0002,0.001,0,0,0]
26.8266
~[1.4745,0.7555,1.4314,0,0,0]
2.0325
~[0.011946,0.004121,0.013409,0,0,0]
0.6075
~[0.002962,0.000618,0.003213,0,0,0]
0.6075
~[0.002962,0.000618,0.003213,0,0,0]
0.4575
~[0.000892,0.000547,0.00134,0,0,0]
2.0325
~[0.011946,0.004121,0.013409,0,0,0]
0.6075
~[0.002962,0.000618,0.003213,0,0,0]
0.6075
~[0.002962,0.000618,0.003213,0,0,0]
0.4575
~[0.000892,0.000547,0.00134,0,0,0]
<opensim.simulation.Body; proxy of <Swig Object of type 'OpenSim::Body *' at 0x000001B6644FF7E0> >


In [19]:
model.getMuscleList()

<opensim.simulation.MuscleList; proxy of <Swig Object of type 'OpenSim::ComponentList< OpenSim::Muscle const > *' at 0x000001B6644FF630> >

In [ ]:
model = osim.Model("calibrated_Rajagopal_2015.osim")
model.setUseVisualizer(True)
initState = model.initSystem()
print(initState.getY())


finalState = osim.simulate(model, initState, 1.5)

In [50]:
# osim.visualize(model)

AttributeError: module 'opensim' has no attribute 'visualize'

In [1]:
osim.MarkerSet()

NameError: name 'osim' is not defined

In [50]:
model = osim.Model("calibrated_Rajagopal_2015.osim")
initState = model.initSystem()
pelvis = model.getBodySet().get("pelvis")
# pelvis_imu = pelvis.getComponentsList().findComponent("pelvis_imu")
pelvis_imu = pelvis.findComponent("pelvis_imu")

In [51]:
for x in pelvis_imu.getStateVariableNames():
    print(x)

TypeError: 'ArrayStr' object is not iterable

In [59]:
pelvis_imu.getOutput('rotation').getValueAsString(initState)

'\n[0.28095286,0.0015755424,0.95972028]\n[0.95752658,0.067133965,-0.28042088]\n[-0.064871642,0.99774273,0.017352856]\n'

In [87]:
# for x in pelvis_imu.getOutputNames():
#     print(x.getName())
for i in range(7):
    prop = pelvis_imu.getPropertyByIndex(i)
    print(prop.getName())

components
frame_geometry
attached_geometry
WrapObjectSet
socket_parent
translation
orientation


In [91]:
orientation = pelvis_imu.getPropertyByName("orientation")
orientation.toString()

translation = pelvis_imu.getPropertyByName("translation")
translation.toString()

'(-0.0707 0 0)'

In [ ]:
model = osim.Model("calibrated_Rajagopal_2015.osim")
model.setUseVisualizer(True)
initState = model.initSystem()
visualizer = model.getVisualizer()
visualizer.show(initState)

In [5]:
modelBodySet =  model.getBodySet()
for body in modelBodySet:
    body_name = body.getName()
    body = modelBodySet.get(body_name)

    # Extract the IMU component
    body_imu = body.findComponent(body_name + "_imu")
    if body_imu is not None:
        print("Info on", body_name, "IMU")
        imu_orientation = body_imu.getPropertyByName("orientation")
        print("Orientation: ", imu_orientation.toString())

        imu_translation = body_imu.getPropertyByName("translation")
        print("Translation: ", imu_translation.toString())
        print("\n")

    else:
        continue

Info on pelvis IMU
Orientation:  (1.50899 1.286 -0.00560779)
Translation:  (-0.0707 0 0)


Info on femur_r IMU
Orientation:  (-3.0795 -0.245978 -1.61288)
Translation:  (0 -0.17 0)


Info on tibia_r IMU
Orientation:  (3.05575 -0.18932 -1.47235)
Translation:  (0 -0.1867 0)


Info on calcn_r IMU
Orientation:  (1.70731 -0.55594 -1.41374)
Translation:  (0.1 0.03 0)


Info on femur_l IMU
Orientation:  (-0.0416244 0.112898 1.40969)
Translation:  (0 -0.17 0)


Info on tibia_l IMU
Orientation:  (0.103356 0.329539 1.47014)
Translation:  (0 -0.1867 0)


Info on calcn_l IMU
Orientation:  (1.53003 -0.518915 1.50722)
Translation:  (0.1 0.03 0)


Info on torso IMU
Orientation:  (1.41822 1.24909 0.075145)
Translation:  (-0.03 0.32 0)




In [8]:
pelvis = model.getBodySet().get("pelvis")
body_imu = pelvis.findComponent("pelvis" + "_imu")
imu_orientation = body_imu.getPropertyByName("orientation")
print("Orientation: ", imu_orientation.toString())

Orientation:  (1.50899 1.286 -0.00560779)


In [15]:
storage = osim.Storage("IKResults/ik_MT_012005D6_009-001_orientations.mot")

# tabe = storage.read(, ".mot")

In [23]:
print(storage.getLastStateVector().getData())

<opensim.common.ArrayDouble; proxy of <Swig Object of type 'OpenSim::Array< double > *' at 0x000001FA80E5EF60> >
